# EDA - (ETL final)

Nota: es importante realizar los procesos de ETL en la carpeta ./ETL/ y el de Feature Engineering en la carpeta ./Feature Engineering/ para realizar este paso.

Si bien el archivo se llamna EDA, aquí se desarrollará un ETL final con un último proceso de filtrado de datos preparándolos para el entrenamiento del modelo de ML.

## Importar Librerías

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import sys

# Ignorar advertencias
warnings.filterwarnings('ignore')

### importar funciones personalizadas de ../functions/EDA.py

In [2]:
# Obtener el directorio de trabajo actual
current_dir = os.getcwd()

# Navegar hacia el directorio raíz del proyecto
project_root = os.path.abspath(os.path.join(current_dir, '..'))

# Agregar la ruta del proyecto al sys.path
sys.path.append(project_root)

In [3]:
# importar función personalizada de ./VideoGameRecommender/functions/EDA.py
# Esta función permite traer el archivo en el formato que se encuentre (CSV o Parquet)
from functions.EDA import get_file, get_frequency
from functions.ETL import export

## ETL Final

Se realiza el Análisis Exploratorio de Datos en user_items revisando elemento por elemento.

### Revisión general

In [4]:
# utilizamos la función get_file(file_name) importada desde ../functions/EDA.py para traer los datasets
user_items = get_file('user_items')
user_reviews = get_file('user_reviews_sentiment_analysis')
steam_games = get_file('steam_games')

In [5]:
# se verifica si existen user_id en user_reviews que no existan en user_items
missing_user = user_reviews[~user_reviews['user_id'].isin(user_items['user_id'])]
missing_user_quant = get_frequency(missing_user, 'user_id')
print(f'Reviews con usuarios que no figuran en user_items: {len(missing_user_quant)}')
missing_user_quant.head()

Reviews con usuarios que no figuran en user_items: 2845


,user_id,frequency,percentage
0,norsktf,16,0.255346
1,76561198077815717,14,0.223428
2,AkkarinDarkmoor,12,0.191510
3,kiniston69,10,0.159591
4,mshee,10,0.159591


In [6]:
# se filtran los usuarios de user_review que no han hecho

In [7]:
# se hace un merge entre user items y reviews en las columnas item_id y user_id de ambos dataframes
users = pd.merge(user_items, user_reviews, how='inner', on=['item_id', 'user_id'])

# se visualiza y su longitud
print(len(users))
users.head(3)

44892


,user_id,items_count,item_id,playtime_forever,posted,recommend,review
0,76561197970982479,277,22200,271.0,2011-07-15,1,2
1,76561197970982479,277,1250,10006.0,2011-11-05,1,2
2,76561197970982479,277,43110,834.0,2011-04-21,1,2


In [8]:
# obtener tipo de datos del dataframe
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44892 entries, 0 to 44891
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           44892 non-null  object 
 1   items_count       44892 non-null  int64  
 2   item_id           44892 non-null  int64  
 3   playtime_forever  44892 non-null  float64
 4   posted            44892 non-null  object 
 5   recommend         44892 non-null  int64  
 6   review            44892 non-null  int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 2.4+ MB


In [9]:
# se filtran las filas donde playtime_forever sea 0 y recommend sea nulo
rubbish = users[((users['playtime_forever'] == 0) & (users['recommend'].isna()))]

# ver la cantidad de registros y las primeras filas
print(len(rubbish))
rubbish.head(3)

0


,user_id,items_count,item_id,playtime_forever,posted,recommend,review


In [10]:
# se hace un merge para obtener las filas que NO están en rubbish
clean_users = users.merge(rubbish, how='left', indicator=True)

# se seleccionan sólo las filas que en _merge es "left_only"
clean_users = clean_users[clean_users['_merge'] == 'left_only']

# se elimina la columna _merge
clean_users.drop(columns='_merge', inplace=True)

print(len(clean_users))
clean_users.head()

44892


,user_id,items_count,item_id,playtime_forever,posted,recommend,review
0,76561197970982479,277,22200,271.0,2011-07-15,1,2
1,76561197970982479,277,1250,10006.0,2011-11-05,1,2
2,76561197970982479,277,43110,834.0,2011-04-21,1,2
3,js41637,888,227300,551.0,2013-09-08,1,2
4,js41637,888,239030,349.0,2013-11-29,1,2


In [11]:
#contar cuántos juegos son por usuario
user_game_count = clean_users[clean_users['item_id'].notna()].groupby('user_id')['item_id'].count()

#mostrar distribución
user_game_count.describe()

count    19913.000000
mean         2.254407
std          1.938736
min          1.000000
25%          1.000000
50%          1.000000
75%          3.000000
max         20.000000
Name: item_id, dtype: float64

### promedio de tiempo de juego y promedio por juego

In [12]:
# Promedio de tiempo de juego por juego
playtime_per_game = clean_users.groupby('item_id')['playtime_forever'].mean()

# Recomendaciones promedio por juego
recommend_per_game = clean_users.groupby('item_id')['recommend'].mean()

# Combinar ambos en un solo DataFrame
game_stats = pd.DataFrame({
    'avg_playtime': playtime_per_game,
    'recommend_rate': recommend_per_game
})

# Ver las primeras filas del nuevo DataFrame
game_stats.head()

,avg_playtime,recommend_rate
item_id,,
10,16621.666667,0.977778
20,4832.800000,0.666667
30,598.000000,0.750000
40,145.000000,1.000000
50,411.333333,0.666667


### Unión con el dataframe steam_games

In [13]:
# se eliminan columnas innecesarias (items_count y posted)
clean_users.drop(columns=['items_count', 'posted'], inplace=True)

clean_users.head()

,user_id,item_id,playtime_forever,recommend,review
0,76561197970982479,22200,271.0,1,2
1,76561197970982479,1250,10006.0,1,2
2,76561197970982479,43110,834.0,1,2
3,js41637,227300,551.0,1,2
4,js41637,239030,349.0,1,2


In [14]:
# ver tipo de datos de clean_users
clean_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44892 entries, 0 to 44891
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           44892 non-null  object 
 1   item_id           44892 non-null  int64  
 2   playtime_forever  44892 non-null  float64
 3   recommend         44892 non-null  int64  
 4   review            44892 non-null  int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 1.7+ MB


In [15]:
# Agrupar por item_id y realizar las agregaciones
grouped = clean_users.groupby('item_id').agg({
    'playtime_forever': 'sum',  # Sumar el tiempo total jugado para cada juego
    'recommend': [ 
        ('positive', lambda x: (x == 1).sum()),  # Contar recomendaciones positivas
        ('negative', lambda x: (x == 0).sum())   # Contar recomendaciones negativas
    ],
    'review': [
        ('negative', lambda x: (x == 0).sum()),  # Contar sentimientos negativos
        ('neutral', lambda x: (x == 1).sum()),   # Contar sentimientos neutrales
        ('positive', lambda x: (x == 2).sum())   # Contar sentimientos positivos
    ]
}).reset_index()

# Aplanar las columnas multinivel
grouped.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in grouped.columns]

# Renombrar columnas (si es necesario)
grouped.rename(columns={
    'item_id_': 'id',
    'playtime_forever_sum': 'playtime_total',
    'recommend_positive': 'recommend_pos', 
    'recommend_negative': 'recommend_neg',
    'review_negative': 'review_neg',
    'review_neutral': 'review_neu',
    'review_positive': 'review_pos',
}, inplace=True)

# Imprimir el DataFrame resultante y su longitud
print(len(grouped))
grouped.head()


3097


,id,playtime_total,recommend_pos,recommend_neg,review_neg,review_neu,review_pos
0,10,747975.0,44,1,0,3,42
1,20,72492.0,10,5,2,3,10
2,30,2392.0,3,1,0,1,3
3,40,145.0,1,0,0,0,1
4,50,1234.0,2,1,0,1,2


In [16]:
# se crea un df final y se visualiza
df = pd.merge(steam_games, grouped, on='id', how='left')

print(len(df))
df.head()

161208


,id,app_name,genres,specs,price,developer,release_year,playtime_total,recommend_pos,recommend_neg,review_neg,review_neu,review_pos
0,761140,Lost Summoner Kitty,indie,single-player,4.99,kotoshiro,2018.0,NaN,NaN,NaN,NaN,NaN,NaN
1,761140,Lost Summoner Kitty,action,single-player,4.99,kotoshiro,2018.0,NaN,NaN,NaN,NaN,NaN,NaN
2,761140,Lost Summoner Kitty,casual,single-player,4.99,kotoshiro,2018.0,NaN,NaN,NaN,NaN,NaN,NaN
3,761140,Lost Summoner Kitty,simulation,single-player,4.99,kotoshiro,2018.0,NaN,NaN,NaN,NaN,NaN,NaN
4,761140,Lost Summoner Kitty,strategy,single-player,4.99,kotoshiro,2018.0,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
#contar nulos
print(len(df))
df.isnull().sum()

161208


id                     0
app_name               0
genres                 2
specs               2228
price                 52
developer              0
release_year         406
playtime_total    125527
recommend_pos     125527
recommend_neg     125527
review_neg        125527
review_neu        125527
review_pos        125527
dtype: int64

In [19]:
# Render pide recortes y no me queda de otra
df.dropna(inplace=True)
print(len(df))
df.head()

35249


,id,app_name,genres,specs,price,developer,release_year,playtime_total,recommend_pos,recommend_neg,review_neg,review_neu,review_pos
210,282010,Carmageddon Max Pack,classic,multi-player,9.99,stainless,1997.0,466.0,1.0,0.0,0.0,0.0,1.0
211,282010,Carmageddon Max Pack,classic,single-player,9.99,stainless,1997.0,466.0,1.0,0.0,0.0,0.0,1.0
212,282010,Carmageddon Max Pack,indie,multi-player,9.99,stainless,1997.0,466.0,1.0,0.0,0.0,0.0,1.0
213,282010,Carmageddon Max Pack,indie,single-player,9.99,stainless,1997.0,466.0,1.0,0.0,0.0,0.0,1.0
214,282010,Carmageddon Max Pack,action,multi-player,9.99,stainless,1997.0,466.0,1.0,0.0,0.0,0.0,1.0


In [20]:
#se reemplazan valores vacíos numéricos y de texto
df['playtime_total'].fillna(0, inplace=True)
df['recommend_pos'].fillna(0, inplace=True)
df['recommend_neg'].fillna(0, inplace=True)
df['review_neg'].fillna(0, inplace=True)
df['review_neu'].fillna(0, inplace=True)
df['review_pos'].fillna(0, inplace=True)

#Eliminar valores nulos en la columna year
df.dropna(subset=['release_year'], inplace=True)

# se muestran los nulos
print(len(df))
df.isnull().sum()

35249


id                0
app_name          0
genres            0
specs             0
price             0
developer         0
release_year      0
playtime_total    0
recommend_pos     0
recommend_neg     0
review_neg        0
review_neu        0
review_pos        0
dtype: int64

### Tratamiento de los precios

Para los precios se realizará el siguiente proceso:
- se calcula la mediana de los precios de las distintas combinaciones de developer, specs y genres
- dado que quedan precios diferentes para el mismo juego al variar las combinaciones, los precios de los juegos con el mismo id se promedian y se redondean a dos decimales.

In [21]:
#agrupar las combinaciones de genres y specs y calcular la mediana de los precios
gs_catprice = df.groupby(['developer', 'genres', 'specs']).agg({
    'price': 'median' # esto permite que el resultado no se vea afectado por outliers
}).reset_index()

# se visualizan columnas y longitud
print(len(gs_catprice))
gs_catprice.head()

27616


,developer,genres,specs,price
0,07thexpansion,adventure,single-player,5.99
1,07thexpansion,anime,single-player,5.99
2,07thexpansion,casual,single-player,5.99
3,07thexpansion,classic,single-player,5.99
4,07thexpansion,dark,single-player,5.99


In [23]:
#agrupar las combinaciones de genres y calcular la mediana de los precios (en caso que specs sea nulo)
g_catprice = df.groupby(['developer', 'genres']).agg({
    'price': 'median' # esto permite que el resultado no se vea afectado por outliers
}).reset_index()

# se visualizan columnas y longitud
print(len(g_catprice))
g_catprice.head()

17722


,developer,genres,price
0,07thexpansion,adventure,5.99
1,07thexpansion,anime,5.99
2,07thexpansion,casual,5.99
3,07thexpansion,classic,5.99
4,07thexpansion,dark,5.99


In [24]:
#agrupar las combinaciones de specs y calcular la mediana de los precios (en caso que genres sea nulo)
s_catprice = df.groupby(['developer', 'specs']).agg({
    'price': 'median' # esto permite que el resultado no se vea afectado por outliers
}).reset_index()

# se visualizan columnas y longitud
print(len(s_catprice))
s_catprice.head()

2872


,developer,specs,price
0,07thexpansion,single-player,5.99
1,10thart,single-player,6.99
2,10tons,single-player,13.99
3,11bit,cross-platform multiplayer,14.99
4,11bit,multi-player,14.99


In [25]:
#agrupar las combinaciones de specs y calcular la mediana de los precios (en caso que genres sea nulo)
dev_catprice = df.groupby(['developer']).agg({
    'price': 'median' # esto permite que el resultado no se vea afectado por outliers
}).reset_index()

# se visualizan columnas y longitud
print(len(dev_catprice))
dev_catprice.head()

1905


,developer,price
0,07thexpansion,5.99
1,10thart,6.99
2,10tons,13.99
3,11bit,14.99
4,14°east,9.99


In [26]:
# Merge del dataframe original con el de medianas para obtener las medianas por combinación
merged_df = df.merge(gs_catprice, on=['developer', 'genres', 'specs'], how='left')

# visualizar
merged_df.head()

,id,app_name,genres,specs,price_x,developer,release_year,playtime_total,recommend_pos,recommend_neg,review_neg,review_neu,review_pos,price_y
0,282010,Carmageddon Max Pack,classic,multi-player,9.99,stainless,1997.0,466.0,1.0,0.0,0.0,0.0,1.0,9.990
1,282010,Carmageddon Max Pack,classic,single-player,9.99,stainless,1997.0,466.0,1.0,0.0,0.0,0.0,1.0,9.990
2,282010,Carmageddon Max Pack,indie,multi-player,9.99,stainless,1997.0,466.0,1.0,0.0,0.0,0.0,1.0,9.990
3,282010,Carmageddon Max Pack,indie,single-player,9.99,stainless,1997.0,466.0,1.0,0.0,0.0,0.0,1.0,9.990
4,282010,Carmageddon Max Pack,action,multi-player,9.99,stainless,1997.0,466.0,1.0,0.0,0.0,0.0,1.0,4.995


In [27]:
# Llenar los valores nulos con las medianas correspondientes
merged_df['price_y'] = merged_df['price_y'].fillna(
    merged_df.merge(s_catprice, on=['developer', 'specs'], how='left')['price_y']
)

merged_df['price_y'] = merged_df['price_y'].fillna(
    merged_df.merge(g_catprice, on=['developer', 'genres'], how='left')['price_y']
)

merged_df['price_y'] = merged_df['price_y'].fillna(
    merged_df.merge(dev_catprice, on='developer', how='left')['price_y']
)

In [28]:
# agrupar por id para calcular la media del precio
final_price = merged_df.groupby('id').agg({
    'price_y': 'mean'  # Media de las combinaciones de géneros y specs
}).reset_index()

final_price.head()

,id,price_y
0,10,12.156667
1,20,9.656667
2,30,9.762727
3,40,8.323333
4,50,14.115000


In [29]:
# crear el df final y ver primeros resultados
df_final = df.merge(final_price, on='id', how='left')

df_final.head()

,id,app_name,genres,specs,price,developer,release_year,playtime_total,recommend_pos,recommend_neg,review_neg,review_neu,review_pos,price_y
0,282010,Carmageddon Max Pack,classic,multi-player,9.99,stainless,1997.0,466.0,1.0,0.0,0.0,0.0,1.0,8.991
1,282010,Carmageddon Max Pack,classic,single-player,9.99,stainless,1997.0,466.0,1.0,0.0,0.0,0.0,1.0,8.991
2,282010,Carmageddon Max Pack,indie,multi-player,9.99,stainless,1997.0,466.0,1.0,0.0,0.0,0.0,1.0,8.991
3,282010,Carmageddon Max Pack,indie,single-player,9.99,stainless,1997.0,466.0,1.0,0.0,0.0,0.0,1.0,8.991
4,282010,Carmageddon Max Pack,action,multi-player,9.99,stainless,1997.0,466.0,1.0,0.0,0.0,0.0,1.0,8.991


In [30]:
# reemplazar valores vacíos de price por los elementos de price_y
df_final['price'] = df_final['price'].fillna(df_final['price_y']).round(2)

# eliminar columna price_y y visualizar

df_final.drop(columns='price_y', inplace=True)

df_final.head()

,id,app_name,genres,specs,price,developer,release_year,playtime_total,recommend_pos,recommend_neg,review_neg,review_neu,review_pos
0,282010,Carmageddon Max Pack,classic,multi-player,9.99,stainless,1997.0,466.0,1.0,0.0,0.0,0.0,1.0
1,282010,Carmageddon Max Pack,classic,single-player,9.99,stainless,1997.0,466.0,1.0,0.0,0.0,0.0,1.0
2,282010,Carmageddon Max Pack,indie,multi-player,9.99,stainless,1997.0,466.0,1.0,0.0,0.0,0.0,1.0
3,282010,Carmageddon Max Pack,indie,single-player,9.99,stainless,1997.0,466.0,1.0,0.0,0.0,0.0,1.0
4,282010,Carmageddon Max Pack,action,multi-player,9.99,stainless,1997.0,466.0,1.0,0.0,0.0,0.0,1.0


### Exportar dataframe final

In [31]:
# se utiliza la función export de ../functions/ETL.py para exportar el archivo final
export(df_final, project_root, 'games')

Archivos exportados exitosamente.
